# **Convolutional Neural Network (CNN) Architecture**

Model definition, build and configuration



In [ ]:
# CNN configuration parameters
class TCNNConfig(object):

    embedding_dim = 128  # Word vector dimension
    seq_length = max_length  # Sequence length (Taken from preprocessing)
    num_classes = num_classes # Number of classes (Taken from preprocessing)
    num_filters = 256  # Number of convolution kernels
    kernel_size = 5  # Convolution kernel size
    vocab_size = len(word_index) + 1 # Small vocabulary expression (Taken from tokenizer)

    hidden_dim = 128  # Fully connected layer neurons

    dropout_keep_prob = 0.5  # Dropout retentio ratio
    learning_rate = 1e-3

    batch_size = 64  # Training size per batch
    num_epochs = 10

    print_per_batch = 100
    save_per_batch = 10

# CNN model build
class TextCNN(tf.keras.Model):

    def __init__(self, config):
        super(TextCNN, self).__init__()
        self.config = config

        self.embedding = tf.keras.layers.Embedding(self.config.vocab_size, self.config.embedding_dim)
        self.conv1 = tf.keras.layers.Conv1D(self.config.num_filters, self.config.kernel_size, activation='relu')
        self.pool1 = tf.keras.layers.GlobalMaxPooling1D()
        self.fc1 = tf.keras.layers.Dense(self.config.hidden_dim, activation='relu')
        self.dropout = tf.keras.layers.Dropout(self.config.dropout_keep_prob)
        self.fc2 = tf.keras.layers.Dense(self.config.num_classes, activation='softmax')

    def call(self, inputs):
        embedding_inputs = self.embedding(inputs)
        conv = self.conv1(embedding_inputs)
        gmp = self.pool1(conv)
        fc = self.fc1(gmp)
        fc = self.dropout(fc)
        logits = self.fc2(fc)
        return logits

In [ ]:
# Model Training
config = TCNNConfig()
model = TextCNN(config)

optimizer = tf.keras.optimizers.Adam(config.learning_rate)
loss_func = tf.keras.losses.CategoricalCrossentropy()

def train_step(inputs, labels):
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = loss_func(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    accuracy = np.mean(np.argmax(labels, axis=1) == np.argmax(predictions, axis=1))
    return loss, accuracy

def evaluate_model(inputs, labels):
    predictions = model(inputs)
    loss = loss_func(labels, predictions)
    accuracy = np.mean(np.argmax(labels, axis=1) == np.argmax(predictions, axis=1))
    return loss, accuracy

epochs = config.num_epochs
batch_size = config.batch_size
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_padded, y_train_one_hot)).shuffle(len(X_train_padded)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val_padded, y_val_one_hot)).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test_padded, y_test_one_hot)).batch(batch_size)

# Initialize lists to store losses and accuracies
train_losses_cnn = []
val_losses_cnn = []
train_accuracies_cnn = []
val_accuracies_cnn = []

print("Training the model...")
for epoch in range(epochs):
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.Mean()
    for batch, (inputs, labels) in enumerate(train_dataset):
        loss, accuracy = train_step(inputs, labels)
        epoch_loss_avg.update_state(loss)
        epoch_accuracy.update_state(accuracy)
        if batch % config.print_per_batch == 0:
            print(f"Epoch {epoch+1}/{epochs} Batch {batch} Loss: {epoch_loss_avg.result():.4f} Accuracy: {epoch_accuracy.result():.4f}")

    # Append epoch losses and accuracies to the lists
    train_losses_cnn.append(epoch_loss_avg.result().numpy())
    train_accuracies_cnn.append(epoch_accuracy.result().numpy())

    val_loss_avg = tf.keras.metrics.Mean()
    val_accuracy = tf.keras.metrics.Mean()
    for inputs, labels in val_dataset:
        loss, accuracy = evaluate_model(inputs, labels)
        val_loss_avg.update_state(loss)
        val_accuracy.update_state(accuracy)

    # Append epoch losses and accuracies to the lists
    val_losses_cnn.append(val_loss_avg.result().numpy())
    val_accuracies_cnn.append(val_accuracy.result().numpy())

    print(f"Epoch {epoch+1}/{epochs} Validation Loss: {val_loss_avg.result():.4f} Validation Accuracy: {val_accuracy.result():.4f}")

Training the model...
Epoch 1/10 Batch 0 Loss: 1.4144 Accuracy: 0.1719
Epoch 1/10 Batch 100 Loss: 1.0256 Accuracy: 0.5702
Epoch 1/10 Batch 200 Loss: 0.7079 Accuracy: 0.7219
Epoch 1/10 Batch 300 Loss: 0.5849 Accuracy: 0.7772
Epoch 1/10 Batch 400 Loss: 0.5171 Accuracy: 0.8061
Epoch 1/10 Batch 500 Loss: 0.4760 Accuracy: 0.8241
Epoch 1/10 Batch 600 Loss: 0.4463 Accuracy: 0.8363
Epoch 1/10 Batch 700 Loss: 0.4244 Accuracy: 0.8455
Epoch 1/10 Batch 800 Loss: 0.4098 Accuracy: 0.8515
Epoch 1/10 Batch 900 Loss: 0.3941 Accuracy: 0.8579
Epoch 1/10 Batch 1000 Loss: 0.3831 Accuracy: 0.8626
Epoch 1/10 Batch 1100 Loss: 0.3731 Accuracy: 0.8666
Epoch 1/10 Batch 1200 Loss: 0.3642 Accuracy: 0.8700
Epoch 1/10 Batch 1300 Loss: 0.3584 Accuracy: 0.8721
Epoch 1/10 Batch 1400 Loss: 0.3511 Accuracy: 0.8751
Epoch 1/10 Validation Loss: 0.2636 Validation Accuracy: 0.9103
Epoch 2/10 Batch 0 Loss: 0.2547 Accuracy: 0.9219
Epoch 2/10 Batch 100 Loss: 0.1947 Accuracy: 0.9313
Epoch 2/10 Batch 200 Loss: 0.1975 Accuracy: 0.9

In [ ]:
model.summary()

Model: "text_cnn_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     multiple                  8872320   
                                                                 
 conv1d_2 (Conv1D)           multiple                  164096    
                                                                 
 global_max_pooling1d_2 (Gl  multiple                  0         
 obalMaxPooling1D)                                               
                                                                 
 dense_4 (Dense)             multiple                  32896     
                                                                 
 dropout_2 (Dropout)         multiple                  0         
                                                                 
 dense_5 (Dense)             multiple                  516       
                                                        

In [ ]:
print("\nEvaluating on the test set...")
test_loss_avg = tf.keras.metrics.Mean()
test_accuracy = tf.keras.metrics.Mean()
for inputs, labels in test_dataset:
    loss, accuracy = evaluate_model(inputs, labels)
    test_loss_avg.update_state(loss)
    test_accuracy.update_state(accuracy)
print(f"Test Loss: {test_loss_avg.result():.4f} Test Accuracy: {test_accuracy.result():.4f}")


Evaluating on the test set...
Test Loss: 0.6398 Test Accuracy: 0.9026
